In [1]:
!pip install cvxpy

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import math
import cvxpy as cp

# Complete the following functions
def strat_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
   x_optimal = x_init
   cash_optimal = cash_init
   return x_optimal, cash_optimal

def strat_equally_weighted(x_init, cash_init, mu, Q, cur_prices):
    
    #xinit curr_positions
    #cash init, left cash, maybe 0 or positive
    # cur_prices  first trading day of each 2-month
    # get the price of each stock

    total_cash= np.dot(cur_prices, x_init)+cash_init
    
#     weight_new=1/n

#     x_optimal= weight_new*total_cash/cur_prices

    money_allc=np.ones((20))*total_cash/20
    x_optimal=np.floor(money_allc/cur_prices)
    #x_optimal= np.floor(x_optimal) # all the stock -1 is used to pay the trasaction fee

    trac_fee= np.dot(abs(x_optimal-x_init),cur_prices )* 0.005

    cash_optimal=total_cash-trac_fee- np.dot(x_optimal ,cur_prices) 
        
    return x_optimal, cash_optimal

def strat_min_variance(x_init, cash_init, mu, Q, cur_prices):
    total_cash= np.dot(cur_prices, x_init)+cash_init
    
    n=20
    w1=cp.Variable(n)
    prob1=cp.Problem(cp.Minimize(cp.quad_form(w1,Q)),[sum(w1)==1,w1>=0])
    prob1.solve(solver=cp.CPLEX,verbose=False,cplex_params={"qpmethod":6})
    w_minVar=w1.value
    
    # new position
    x_optimal= np.floor(w_minVar*total_cash/cur_prices)
    
  
    trac_fee=np.dot(abs(x_optimal-x_init),cur_prices ) *0.005

    cash_optimal=total_cash-trac_fee-np.dot(x_optimal,cur_prices)
    return x_optimal, cash_optimal

def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    x_optimal = x_init
    cash_optimal = cash_init
    return x_optimal, cash_optimal
# Input file
input_file_prices = 'Daily_closing_prices.csv'

# Read data into a dataframe
df = pd.read_csv(input_file_prices)

# Convert dates into array [year month day]
def convert_date_to_array(datestr):
    temp = [int(x) for x in datestr.split('/')]
    return [temp[-1], temp[0], temp[1]]

dates_array = np.array(list(df['Date'].apply(convert_date_to_array)))
data_prices = df.iloc[:, 1:].to_numpy()
dates = np.array(df['Date'])
# Find the number of trading days in Nov-Dec 2014 and
# compute expected return and covariance matrix for period 1
day_ind_start0 = 0
day_ind_end0 = len(np.where(dates_array[:,0]==2014)[0])
cur_returns0 = data_prices[day_ind_start0+1:day_ind_end0,:] / data_prices[day_ind_start0:day_ind_end0-1,:] - 1
mu = np.mean(cur_returns0, axis = 0)
Q = np.cov(cur_returns0.T)

# Remove datapoints for year 2014
data_prices = data_prices[day_ind_end0:,:]
dates_array = dates_array[day_ind_end0:,:]
dates = dates[day_ind_end0:]

# Initial positions in the portfolio
init_positions = np.array([5000, 950, 2000, 0, 0, 0, 0, 2000, 3000, 1500, 0, 0, 0, 0, 0, 0, 1001, 0, 0, 0])

# Initial value of the portfolio
init_value = np.dot(data_prices[0,:], init_positions)
print('\nInitial portfolio value = $ {}\n'.format(round(init_value, 2)))

# Initial portfolio weights
w_init = (data_prices[0,:] * init_positions) / init_value

# Number of periods, assets, trading days
N_periods = 6*len(np.unique(dates_array[:,0])) # 6 periods per year
N = len(df.columns)-1
N_days = len(dates)

# Annual risk-free rate for years 2015-2016 is 2.5%
r_rf = 0.025

# Number of strategies
strategy_functions = ['strat_buy_and_hold', 'strat_equally_weighted', 'strat_min_variance', 'strat_max_Sharpe']
strategy_names     = ['Buy and Hold', 'Equally Weighted Portfolio', 'Mininum Variance Portfolio', 'Maximum Sharpe Ratio Portfolio']
#N_strat = 1  # comment this in your code
N_strat = len(strategy_functions)  # uncomment this in your code
fh_array = [strat_buy_and_hold, strat_equally_weighted, strat_min_variance, strat_max_Sharpe]

portf_value = [0] * N_strat
x = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
cash = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
for period in range(1, N_periods+1):
   # Compute current year and month, first and last day of the period
   if dates_array[0, 0] == 15:
       cur_year  = 15 + math.floor(period/7)
   else:
       cur_year  = 2015 + math.floor(period/7)

   cur_month = 2*((period-1)%6) + 1
   day_ind_start = min([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month)) if val])
   day_ind_end = max([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month+1)) if val])
   print('\nPeriod {0}: start date {1}, end date {2}'.format(period, dates[day_ind_start], dates[day_ind_end]))
   
   # Prices for the current day
   cur_prices = data_prices[day_ind_start,:]

   # Execute portfolio selection strategies
   for strategy  in range(N_strat):

      # Get current portfolio positions
      if period == 1:
         curr_positions = init_positions
         curr_cash = 0
         portf_value[strategy] = np.zeros((N_days, 1))
      else:
         curr_positions = x[strategy, period-2]
         curr_cash = cash[strategy, period-2]

      # Compute strategy
      x[strategy, period-1], cash[strategy, period-1] = fh_array[strategy](curr_positions, curr_cash, mu, Q, cur_prices)

      # Verify that strategy is feasible (you have enough budget to re-balance portfolio)
      # Check that cash account is >= 0
      # Check that we can buy new portfolio subject to transaction costs

      ###################### Insert your code here ############################
      if cash[strategy][period-1] < 0:
          
          total_cash=np.dot(cur_prices, curr_positions)+curr_cash
          
          ratio = x[strategy][period-1] / sum(x[strategy][period-1])
          cash_balance = abs(cash[strategy][period-1])* ratio
          position_balance = np.ceil(cash_balance / cur_prices)

          x[strategy][period-1] = x[strategy][period-1] - position_balance
          transaction = np.dot(cur_prices , abs(x[strategy][period-1] - curr_positions) )* 0.005
          cash[strategy][period-1] = total_cash - np.dot(cur_prices , x[strategy][period-1]) - transaction


      # Compute portfolio value
      p_values = np.dot(data_prices[day_ind_start:day_ind_end+1,:], x[strategy, period-1]) + cash[strategy, period-1]
      portf_value[strategy][day_ind_start:day_ind_end+1] = np.reshape(p_values, (p_values.size,1))
      print('  Strategy "{0}", value begin = $ {1:.2f}, value end = $ {2:.2f}'.format( strategy_names[strategy], 
             portf_value[strategy][day_ind_start][0], portf_value[strategy][day_ind_end][0]))

      
   # Compute expected returns and covariances for the next period
   cur_returns = data_prices[day_ind_start+1:day_ind_end+1,:] / data_prices[day_ind_start:day_ind_end,:] - 1
   mu = np.mean(cur_returns, axis = 0)
   Q = np.cov(cur_returns.T)

# Plot results
###################### Insert your code here ############################


Initial portfolio value = $ 1000002.12


Period 1: start date 1/2/2015, end date 2/27/2015
  Strategy "Buy and Hold", value begin = $ 1000002.12, value end = $ 1043785.08
  Strategy "Equally Weighted Portfolio", value begin = $ 992880.88, value end = $ 1020038.79
  Strategy "Mininum Variance Portfolio", value begin = $ 991454.43, value end = $ 1016159.03
  Strategy "Maximum Sharpe Ratio Portfolio", value begin = $ 1000002.12, value end = $ 1043785.08

Period 2: start date 3/2/2015, end date 4/30/2015
  Strategy "Buy and Hold", value begin = $ 1045234.09, value end = $ 1069877.19
  Strategy "Equally Weighted Portfolio", value begin = $ 1030724.21, value end = $ 1011178.46
  Strategy "Mininum Variance Portfolio", value begin = $ 1023391.23, value end = $ 1014243.70
  Strategy "Maximum Sharpe Ratio Portfolio", value begin = $ 1045234.09, value end = $ 1069877.19

Period 3: start date 5/1/2015, end date 6/30/2015
  Strategy "Buy and Hold", value begin = $ 1085647.24, value end = $ 1027659

In [0]:
#df

In [0]:
dates_array[0]

In [0]:
mu

In [0]:
Q

In [0]:
data_prices[0]


In [0]:
day_ind_end0

In [0]:
import numpy as np
## what is the difference???? keep this question
a = np.array([9, 5, 7, 4])
c = np.array([9, 5, 4, 3])
print(a,c)
print(np.dot(a, c))

print(np.matmul(a, c))

In [0]:
np.floor(10/a)

In [0]:
10/a

In [0]:
import cplex 